## Prasyarat

- Python versi 3.8
- Tensorflow dan Tensorflow-GPU (optional) harus memiliki versi yang sama
- CUDNN sesuaikan versi dengan yang diminta tensorflow (optional)

## Install Library

Catatan: Untuk `tensorflow-gpu` dan `cudnn` itu optional. Jika perangkat punya gpu yang support cuda, install library tersebut. Pastikan juga versi tensorflow dan tensorflow gpu sesuai dengan yang diminta oleh tensorflow. Cek di [sini](https://www.tensorflow.org/install/source_windows#gpu)

In [ ]:
# !pip install opencv-python mediapipe scikit-learn matplotlib tensorflow tensoflow-gpu cudnn

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

## Fungsi-Fungsi untuk deteksi keypoint

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                           mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                           mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

## Testing camera

Catatan: Cell ini boleh dijalankan ataupun tidak

In [6]:
cap = cv2.VideoCapture(0)

# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.9, min_tracking_confidence=0.9) as holistic:
    while cap.isOpened():
        # read feed
        ret, frame = cap.read()
        
        # make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # draw landmarks
        draw_styled_landmarks(image, results)

        # show to screen
        cv2.imshow('OpenCV Feed', image)

        # break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

## Fungsi untuk ekstrak keypoint

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # return np.concatenate([pose, face, lh, rh])
    return np.concatenate([pose, lh, rh])

## Collecting data

- actions = gerakan yang ingin di deteksi
- no_sequences = setiap gerakan akan memiliki sejumlah N video
- sequence_length = setiap video memiliki sejumlah N frame

In [12]:
DATA_PATH = os.path.join('dataset')

# actions that we try to detect
# actions = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 
#                     'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 
#                     'w', 'x', 'y', 'z'])
actions = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 's', 't', 'u', 'v', 
                    'w', 'x', 'y', 'z'])

# 30 videos for data
no_sequences = 30

# 30 frames in length
sequence_length = 30

In [13]:
for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## Catatan

Untuk menghentikan pengambilan gambar, tekan tombol `q` pada keyboard, buat cell baru dengan kode 
```python 
cap.release()
cv2.destroyAllWindows()
```
Lalu jalankan

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.9, min_tracking_confidence=0.9) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                
                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [15]:
    cap.release()
    cv2.destroyAllWindows()

## Preprocessing atau labeling

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## One-hot encoding pada label

Catatan: Agar label lebih mudah dipahami mesin, lakukan step ini

In [14]:
label_map = {label: num for num, label in enumerate(actions)}

In [15]:
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 's': 8,
 't': 9,
 'u': 10,
 'v': 11,
 'w': 12,
 'x': 13,
 'y': 14,
 'z': 15}

In [16]:
sequences = []
labels = []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [19]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

## Splitting dataset

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # test_size mempengaruhi hasil akurasi

In [21]:
X.shape

(480, 30, 258)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Build and train model

Catatan: Fungsi myCallbak digunakan untuk menghentikan proses training jika tingkat akurasi sudah sesuai dengan kriteria yang sudah ditentukan sebelumnya

In [33]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if((logs.get('loss')<0.3 and logs.get('categorical_accuracy')>0.9) 
           and (logs.get('val_loss')<0.3 and logs.get('val_categorical_accuracy')>0.9)):
            print("\nTarget akurasi tercapai")
            self.model.stop_training = True

In [34]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [35]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
Total params: 237,680
Trainable params: 237,680
Non-tr

In [37]:
callback = myCallback()
model.fit(X_train, y_train, epochs=2000, callbacks=[callback], validation_data=(X_test, y_test)) # Sesuaikan jumlah epochs
# Cek lagi hasilnya, jangan sampai overfitting atau underfitting
# Untuk kondisi sekarang: Overfitting karena data terlalu sedikit yang mengakibatkan akurasi training tinggi, tapi testing rendah

Epoch 1/2000
12/12 [==============================] - 5s 102ms/step - loss: 2.7660 - categorical_accuracy: 0.0703 - val_loss: 2.6705 - val_categorical_accuracy: 0.1562
Epoch 2/2000
12/12 [==============================] - 1s 54ms/step - loss: 2.6584 - categorical_accuracy: 0.1146 - val_loss: 2.6081 - val_categorical_accuracy: 0.1250
Epoch 3/2000
12/12 [==============================] - 1s 53ms/step - loss: 2.5915 - categorical_accuracy: 0.1380 - val_loss: 2.5364 - val_categorical_accuracy: 0.1667
Epoch 4/2000
12/12 [==============================] - 1s 54ms/step - loss: 2.4417 - categorical_accuracy: 0.1484 - val_loss: 2.2620 - val_categorical_accuracy: 0.1771
Epoch 5/2000
12/12 [==============================] - 1s 54ms/step - loss: 2.3563 - categorical_accuracy: 0.1276 - val_loss: 2.1905 - val_categorical_accuracy: 0.1979
Epoch 6/2000
12/12 [==============================] - 1s 54ms/step - loss: 2.2225 - categorical_accuracy: 0.1667 - val_loss: 2.1182 - val_categorical_accuracy: 0.15

Epoch 50/2000
12/12 [==============================] - 1s 54ms/step - loss: 0.8483 - categorical_accuracy: 0.6432 - val_loss: 0.7441 - val_categorical_accuracy: 0.6458
Epoch 51/2000
12/12 [==============================] - 1s 55ms/step - loss: 1.0641 - categorical_accuracy: 0.6302 - val_loss: 1.1684 - val_categorical_accuracy: 0.5625
Epoch 52/2000
12/12 [==============================] - 1s 53ms/step - loss: 1.0783 - categorical_accuracy: 0.5938 - val_loss: 1.0742 - val_categorical_accuracy: 0.6042
Epoch 53/2000
12/12 [==============================] - 1s 55ms/step - loss: 0.9073 - categorical_accuracy: 0.6484 - val_loss: 0.8528 - val_categorical_accuracy: 0.6354
Epoch 54/2000
12/12 [==============================] - 1s 55ms/step - loss: 0.7604 - categorical_accuracy: 0.6745 - val_loss: 0.7997 - val_categorical_accuracy: 0.6250
Epoch 55/2000
12/12 [==============================] - 1s 56ms/step - loss: 0.6944 - categorical_accuracy: 0.6823 - val_loss: 0.7057 - val_categorical_accuracy:

Epoch 99/2000
12/12 [==============================] - 1s 54ms/step - loss: 0.7039 - categorical_accuracy: 0.7005 - val_loss: 0.6867 - val_categorical_accuracy: 0.7188
Epoch 100/2000
12/12 [==============================] - 1s 54ms/step - loss: 1.0524 - categorical_accuracy: 0.5938 - val_loss: 0.9582 - val_categorical_accuracy: 0.5625
Epoch 101/2000
12/12 [==============================] - 1s 53ms/step - loss: 1.0336 - categorical_accuracy: 0.6120 - val_loss: 1.4514 - val_categorical_accuracy: 0.4271
Epoch 102/2000
12/12 [==============================] - 1s 53ms/step - loss: 1.1775 - categorical_accuracy: 0.5156 - val_loss: 1.1042 - val_categorical_accuracy: 0.5625
Epoch 103/2000
12/12 [==============================] - 1s 53ms/step - loss: 0.8686 - categorical_accuracy: 0.6589 - val_loss: 1.1691 - val_categorical_accuracy: 0.6146
Epoch 104/2000
12/12 [==============================] - 1s 54ms/step - loss: 0.7657 - categorical_accuracy: 0.6979 - val_loss: 0.9611 - val_categorical_accu

12/12 [==============================] - 1s 54ms/step - loss: 0.8593 - categorical_accuracy: 0.6641 - val_loss: 0.7584 - val_categorical_accuracy: 0.8125
Epoch 148/2000
12/12 [==============================] - 1s 54ms/step - loss: 0.6221 - categorical_accuracy: 0.7474 - val_loss: 0.5759 - val_categorical_accuracy: 0.7708
Epoch 149/2000
12/12 [==============================] - 1s 53ms/step - loss: 0.5591 - categorical_accuracy: 0.7500 - val_loss: 0.4765 - val_categorical_accuracy: 0.8333
Epoch 150/2000
12/12 [==============================] - 1s 54ms/step - loss: 0.4349 - categorical_accuracy: 0.8333 - val_loss: 0.5151 - val_categorical_accuracy: 0.7812
Epoch 151/2000
12/12 [==============================] - 1s 55ms/step - loss: 0.4153 - categorical_accuracy: 0.8359 - val_loss: 0.4032 - val_categorical_accuracy: 0.8646
Epoch 152/2000
12/12 [==============================] - 1s 53ms/step - loss: 0.3805 - categorical_accuracy: 0.8333 - val_loss: 0.3530 - val_categorical_accuracy: 0.8438
E

## Save model

In [38]:
model.save('test.h5') ## nama file bisa disesuaikan

## Catatan kaki

- Untuk melakukan testing model, buka file `testing.ipynb` lalu jalankan
- Jika ingin menambahkan dataset, buka file `refine_data.ipynb` lalu jalankan